## TFRecords制作

为了高效地读取数据，可以将数据进行序列化存储，这样也便于网络流式读取数据。TFRecord是一种比较常用的存储二进制序列数据的方法

- tf.Example类是一种将数据表示为{"string": value}形式的meassage类型，Tensorflow经常使用tf.Example来写入、读取TFRecord数据

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import tensorflow as tf

### 通常情况下，tf.Example中可以使用以下几种格式：

- tf.train.BytesList: 可以使用的类型包括 string和byte
- tf.train.FloatList: 可以使用的类型包括 float和double
- tf.train.Int64List: 可以使用的类型包括 enum,bool, int32, uint32, int64

转化实例

In [2]:
def _bytes_feature(value):
    """Returns a bytes_list from a string/byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Return a float_list form a float/double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Return a int64_list from a bool/enum/int/uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [3]:
# tf.train.BytesList
print(_bytes_feature(b'test_string'))
print(_bytes_feature('test_string'.encode('utf8')))

# tf.train.FloatList
print(_float_feature(np.exp(1)))

# tf.train.Int64List
print(_int64_feature(True))
print(_int64_feature(1))

bytes_list {
  value: "test_string"
}

bytes_list {
  value: "test_string"
}

float_list {
  value: 2.7182817459106445
}

int64_list {
  value: 1
}

int64_list {
  value: 1
}



### tfrecord制作方法

创建tf.Example

In [4]:
def serialize_example(feature0, feature1, feature2, feature3):
    """
    创建tf.Example
    """
    
    # 转换成相应类型
    feature = {
        'feature0': _int64_feature(feature0),
        'feature1': _int64_feature(feature1),
        'feature2': _bytes_feature(feature2),
        'feature3': _float_feature(feature3),
    }
    #使用tf.train.Example来创建
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    #SerializeToString方法转换为二进制字符串
    return example_proto.SerializeToString()

In [5]:
# 数据量
n_observations = int(1e4)

# Boolean feature
feature0 = np.random.choice([False, True], n_observations)

# Integer feature
feature1 = np.random.randint(0, 5, n_observations)

# String feature
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[feature1]

# Float feature
feature3 = np.random.randn(n_observations)

In [6]:
filename = 'tfrecord-1'

with tf.io.TFRecordWriter(filename) as writer:
    for i in range(n_observations):
        example = serialize_example(feature0[i], feature1[i], feature2[i], feature3[i])
        writer.write(example)

### 加载tfrecord文件

In [7]:
filenames = [filename]

# 读取
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

### 图像数据处理实例
- 取出全部图片
- 设置labels
- 定义核心函数_bytes_feature _float_feature _int64_feature image_example 
- 制作时，先调用image_example得到example,再调用SerializeToString序列化，最后写入.tfrecord文件

In [8]:
import os
import glob
from datetime import datetime

import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [9]:
image_path = '../img/'
images = glob.glob(image_path + '*.jpg')

for fname in images:
    image = mpimg.imread(fname)
    f, (ax1) = plt.subplots(1, 1, figsize=(8,8))
    f.subplots_adjust(hspace = .2, wspace = .05)
    
    ax1.imshow(image)
    ax1.set_title('Image', fontsize=20)

In [10]:
image_labels = {
    'dog': 0,
    'kangaroo': 1,
}

### 制作TFRecord

In [11]:
# 读数据，binary格式
image_string = open('./img/dog.jpg', 'rb').read()
label = image_labels['dog']
print(image_string)

b'\xff\xd8\xff\xe1\x02\xe6Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\n\x01\x0e\x00\x02\x00\x00\x00 \x00\x00\x00\x86\x01\x0f\x00\x02\x00\x00\x00\x06\x00\x00\x00\xa6\x01\x10\x00\x02\x00\x00\x00\x14\x00\x00\x00\xac\x01\x12\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\x01\x1a\x00\x05\x00\x00\x00\x01\x00\x00\x00\xc0\x01\x1b\x00\x05\x00\x00\x00\x01\x00\x00\x00\xc8\x01(\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x011\x00\x02\x00\x00\x00\x0b\x00\x00\x00\xd0\x012\x00\x02\x00\x00\x00\x14\x00\x00\x00\xdc\x87i\x00\x04\x00\x00\x00\x01\x00\x00\x00\xf0\x00\x00\x00\x00                               \x00Canon\x00Canon PowerShot S95\x00\x00\x00\x00H\x00\x00\x00\x01\x00\x00\x00H\x00\x00\x00\x01Photos 1.0\x00\x002014:09:19 16:08:30\x00\x00\x1e\x82\x9a\x00\x05\x00\x00\x00\x01\x00\x00\x02^\x82\x9d\x00\x05\x00\x00\x00\x01\x00\x00\x02f\x88\'\x00\x03\x00\x00\x00\x01\x00\xfa\x00\x00\x880\x00\x03\x00\x00\x00\x01\x00\x04\x00\x00\x90\x00\x00\x07\x00\x00\x00\x040230\x90\x03\x00\x02\x00\x00\x00\x14\x00\x00\x02n\x90\x04\x00\x02

In [12]:
def _bytes_feature(value):
    """Returns a bytes_list from a string/byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Return a float_list form a float/double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Return a int64_list from a bool/enum/int/uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# 创建图像数据的Example
def image_example(image_string, label):
    image_shape = tf.image.decode_jpeg(image_string).shape
    
    print(image_shape)

    feature = {
        'height': _int64_feature(image_shape[0]),
        'width': _int64_feature(image_shape[1]),
        'depth': _int64_feature(image_shape[2]),
        'label': _int64_feature(label),
        'image_raw': _bytes_feature(image_string),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

In [14]:
#打印部分信息
image_example_proto = image_example(image_string, label)

for line in str(image_example_proto).split('\n')[:15]:
    print(line)
print('...')

(576, 768, 3)
features {
  feature {
    key: "depth"
    value {
      int64_list {
        value: 3
      }
    }
  }
  feature {
    key: "height"
    value {
      int64_list {
        value: 576
      }
...


In [16]:
# 制作 `images.tfrecords`.

image_path = './img/'
images = glob.glob(image_path + '*.jpg')
record_file = 'images.tfrecord'
counter = 0

with tf.io.TFRecordWriter(record_file) as writer:
    for fname in images:
        with open(fname, 'rb') as f:
            image_string = f.read()
            label = image_labels[os.path.basename(fname).replace('.jpg', '')]
            
            # `tf.Example` 
            tf_example = image_example(image_string, label)
            
            # 将`tf.example` 写入 TFRecord 
            writer.write(tf_example.SerializeToString())
            
            counter += 1
            print('Processed {:d} of {:d} images.'.format(
                counter, len(images)))

print(' Wrote {} images to {}'.format(counter, record_file))

(576, 768, 3)
Processed 1 of 2 images.
(600, 900, 3)
Processed 2 of 2 images.
 Wrote 2 images to images.tfrecord


### 加载制作好的TFRecord
- 先用tf.data.TFRecordDataset(tfrecord_file) 加载数据 得到raw_train_dataset
- 定义解析格式
- 定义按照解析格式解析单个example的方法parse_tf_example 注意数据预处理的恢复
- map方法里传入解析方法，raw_train_dataset.map(parse_tf_example) 得到  x_train, y_train
- 制作训练集时用shuffle batch repeate

In [38]:
raw_train_dataset = tf.data.TFRecordDataset('images.tfrecord')
raw_train_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

example数据都进行了序列化，还需要解析以下之前写入的序列化string
- tf.io.parse_single_example(example_proto, feature_description)函数可以解析单条example

In [39]:
# 解析的格式需要跟之前创建example时一致
image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

现在看起来仅仅完成了一个样本的解析，实际数据不可能一个个来写吧，可以定义一个映射规则map函数

In [40]:
def parse_tf_example(example_proto):
    # 解析出来
    parsed_example = tf.io.parse_single_example(example_proto, image_feature_description)
    
    # 预处理
    x_train = tf.image.decode_jpeg(parsed_example['image_raw'], channels=3)
    x_train = tf.image.resize(x_train, (416, 416))
    x_train /= 255.

    lebel = parsed_example['label']
    y_train = lebel
    
    return x_train, y_train

train_dataset = raw_train_dataset.map(parse_tf_example)
train_dataset

<MapDataset shapes: ((416, 416, 3), ()), types: (tf.float32, tf.int64)>

### 制作训练集

In [41]:
num_epochs = 10

train_ds = train_dataset.shuffle(buffer_size=10000).batch(2).repeat(num_epochs)
train_ds

<RepeatDataset shapes: ((None, 416, 416, 3), (None,)), types: (tf.float32, tf.int64)>

In [42]:
for batch, (x, y) in enumerate(train_ds):
    print(batch, x.shape, y)

0 (2, 416, 416, 3) tf.Tensor([0 1], shape=(2,), dtype=int64)
1 (2, 416, 416, 3) tf.Tensor([0 1], shape=(2,), dtype=int64)
2 (2, 416, 416, 3) tf.Tensor([0 1], shape=(2,), dtype=int64)
3 (2, 416, 416, 3) tf.Tensor([1 0], shape=(2,), dtype=int64)
4 (2, 416, 416, 3) tf.Tensor([1 0], shape=(2,), dtype=int64)
5 (2, 416, 416, 3) tf.Tensor([1 0], shape=(2,), dtype=int64)
6 (2, 416, 416, 3) tf.Tensor([0 1], shape=(2,), dtype=int64)
7 (2, 416, 416, 3) tf.Tensor([1 0], shape=(2,), dtype=int64)
8 (2, 416, 416, 3) tf.Tensor([1 0], shape=(2,), dtype=int64)
9 (2, 416, 416, 3) tf.Tensor([1 0], shape=(2,), dtype=int64)


In [43]:
model = tf.keras.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(2, activation='softmax')
])

In [44]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [45]:
model.fit(train_ds, epochs=num_epochs)

Epoch 1/10
10/10 [==============================] - 1s 51ms/step - loss: 55.1923 - accuracy: 0.6500
Epoch 2/10
10/10 [==============================] - 0s 20ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
10/10 [==============================] - 0s 19ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 4/10
10/10 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
10/10 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
10/10 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
10/10 [==============================] - 0s 18ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
10/10 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
10/10 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
10/10 [==============================] - 0s 19ms/step - loss: 0.0000